<a href="https://colab.research.google.com/github/Chayansp/Data-Engineer/blob/main/Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract data from MySQL database

**Install PyMySQL**

In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 1.4 MB/s eta 0:00:00


In [ ]:
import os
import pymysql.cursors



```
#MYSQL_HOST = 'HOST'
  MYSQL_PORT = 3306             
  MYSQL_USER = 'USERNAME'
  MYSQL_PASSWORD = 'PASSWORD'
  MYSQL_DB = 'DATABASE_NAME'
  MYSQL_CHARSET = 'utf8mb4'
```



In [ ]:

class Config:

  MYSQL_HOST = 'HOST'

  MYSQL_PORT = 3306 

  MYSQL_USER = 'USER'

  MYSQL_PASSWORD = 'PASSWORD'

  MYSQL_DB = 'DATABASE'

  MYSQL_CHARSET = 'utf8mb4'


In [ ]:
# Connect to the database
connection = pymysql.connect(
                            host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor
)  

In [ ]:
connection

**Show all tables**

In [ ]:
cursor = connection.cursor()
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
cursor.close()  
print(tables)

[{'Tables_in_r2de2': 'audible_data'}, {'Tables_in_r2de2': 'audible_transaction'}]


**Show some table by with statement**

In [ ]:
with connection.cursor() as cursor:
  sql = "SELECT * FROM audible_data LIMIT 5; " 

  cursor.execute(sql)
  result = cursor.fetchall()

  print("number of rows: ",  len(result))

number of rows:  2269


**Convert the raw data into the DataFrame with the Pandas library**

In [ ]:
import pandas as pd

audible_data = pd.DataFrame(result)
audible_data = audible_data.set_index("Book_ID") 

type(audible_data)

pandas.core.frame.DataFrame

In [ ]:
audible_data.head()

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
Book_ID,,,,,,,,,,
1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65


#Join table: audible_transaction & audible_data

In [ ]:
#audible_transaction
sql = "SELECT * FROM audible_transaction"
audible_transaction = pd.read_sql(sql, connection)
#audible_transaction

In [ ]:
transaction = audible_transaction.merge(
    audible_data, how="left", 
    left_on="book_id", 
    right_on="Book_ID"
  )

In [ ]:
transaction.head()

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49


# Get data from REST API

In [ ]:
import requests

In [ ]:
url = "https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate"
response = requests.get(url)
response.status_code
result_conversion_rate = response.json()

In [ ]:
#result_conversion_rate 

**Convert json to DataFrame**

In [ ]:
conversion_rate = pd.DataFrame(result_conversion_rate)

In [ ]:
conversion_rate

,conversion_rate
2021-04-01,31.194
2021-04-02,31.290
2021-04-03,31.256
2021-04-04,31.244
2021-04-05,31.342
...,...
2021-08-08,33.395
2021-08-09,33.464
2021-08-10,33.460
2021-08-11,33.145


**Reset index and rename**

In [ ]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
conversion_rate[:3] 

,date,conversion_rate
0,2021-04-01,31.194
1,2021-04-02,31.290
2,2021-04-03,31.256


#Join transaction from MySQL database and conversion rate from REST API

In [ ]:
#transaction

In [ ]:
#copy timestamp column then change timestamp column name to date column name
transaction['date'] = transaction['timestamp']
transaction.head(3)

In [ ]:
#convert timestamp to date in both transaction and conversion_rate 
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

In [ ]:
#merge transaction and conversion_rate date by left join with date key
final_df = transaction.merge(conversion_rate, how="left", left_on="date", right_on="date")
final_df.head(3)

In [ ]:
final_df["Price"] = final_df.apply(lambda x: x["Price"].replace("$"," "), axis=1 )
final_df["Price"] = final_df["Price"].astype(float)

In [ ]:
final_df.head(3)

In [ ]:
final_df["THBPrice"] = final_df["Price"]*final_df["conversion_rate"]
final_df

In [ ]:
#another solution
def convert_rate(price, rate):
  return price * rate
##.apply()
final_df["THBPrice"] = final_df.apply(lambda x: convert_rate(x["Price"],x["conversion_rate"]),axis=1)

In [ ]:
final_df["THBPrice"].head()

0    1089.9000
1     610.0326
2     980.9100
3     621.2430
4     544.6386
Name: THBPrice, dtype: float64

In [ ]:
#another solution
def convert_rate(price, rate):
  return price * rate
  
final_df["THBPrice"] = final_df.apply(lambda x: x["Price"] * x["conversion_rate"] ,axis=1)
final_df["THBPrice"].head()

0    1089.9000
1     610.0326
2     980.9100
3     621.2430
4     544.6386
Name: THBPrice, dtype: float64

In [ ]:
final_df = final_df.drop("date", axis=1)
final_df.head(3)

#Export data to csv file

In [ ]:
final_df.to_csv("output.csv",index=False)

NameError: ignored

In [ ]:
%%writefile .env   
MYSQL_HOST='localhost'
MYSQL_PORT= 3306
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root'
MYSQL_DB = 'MyDEProject'
MYSQL_CHARSET = 'utf8mb4'